In [1]:
# Import relevant packages
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets


import os
import time
import warnings
warnings.filterwarnings("ignore", category = FutureWarning)

In [2]:
notebook_path = os.path.abspath("confusion_matrix.ipynb")
model_path = os.path.dirname(notebook_path) + '/model.pth'

model = torch.load(model_path)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/shawn/Documents/GitHub/CS221-Project/project/project/model.pth'